In [1]:
import logging
import uuid

from random import sample

from scripts.utils.file_utils import load_json, dump_json
from scripts.ripe_atlas.ping import PING
from scripts.ripe_atlas.atlas_api import get_measurements_from_tag, get_prefix_from_ip
from default import ANCHORS_FILE, HITLIST_FILE, PREFIX_ANCHOR_VP, TARGET_ANCHOR_VP, MEASUREMENT_CONFIG_PATH, RIPE_CREDENTIALS


logging.basicConfig(level=logging.INFO)


NB_TARGETS = 2
NB_VPS = 4

#NBTARGETS = -1
#NB_VPS = -1

In [2]:
# load datasets

targets_per_prefix = load_json(HITLIST_FILE)

anchors = load_json(ANCHORS_FILE)

## select targets and vps dataset

In [20]:
if NB_TARGETS > 0:
    targets = sample(anchors, k=NB_TARGETS)
else:
    targets = anchors

for i in range(len(targets)):
    targets[i] = targets[i]["address_v4"]


vps = {}

if NB_VPS > 0:
    shuffle = sample(anchors, k=NB_VPS)
    for anchor in shuffle:
        vps[anchor["address_v4"]] = anchor
else:
    shuffle = sample(anchors)
    for anchor in shuffle:
        vps[anchor["address_v4"]] = anchor


print(f"nb targets: {len(targets)}")
print(f"nb_vps : {len(vps)}")

INFO:root:nb targets: 2
INFO:root:nb_vps : 4


# Tier 1: probing each target prefixes

## measure target prefixes

In [24]:
dry_run = False
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all anchors as vps",
    "type": "prefix",
    "af": 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
config_file = MEASUREMENT_CONFIG_PATH / f"{measurement_config['UUID']}.json"
dump_json(measurement_config, config_file)

pinger = PING(RIPE_CREDENTIALS)

# get target prefixes
target_prefixes = []
for target_addr in targets:
    target_prefix = get_prefix_from_ip(target_addr)
    target_prefixes.append(target_prefix)

try:
    cached_results = load_json(PREFIX_ANCHOR_VP)
    logging.info(
        f"initial length targets: {len(target_prefixes)}, cached measurements : {len(cached_results)}"
    )

    # get prefixes out of targets
    cached_results = [
        get_prefix_from_ip(target_addr) for target_addr in cached_results
    ]
    target_prefixes = list(
        set(target_prefixes).difference(set(cached_results)))

    logging.info(
        f"after removing cached: {len(target_prefixes)}, cached measurements : {len(cached_results)}"
    )
except FileNotFoundError:
    logging.info("No cached results available")
    pass

logging.info(
        f"Starting measurements {str(measurement_uuid)} with parameters: dry_run={dry_run}; nb_targets={len(target_prefixes)}; nb_vps={len(vps)}."
    )

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = pinger.ping_by_prefix(
    target_prefixes=target_prefixes,
    vps=vps,
    targets_per_prefix=targets_per_prefix,
    tag=measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration after measurements
dump_json(measurement_config, config_file)

INFO:root:No cached results available
INFO:root:Starting measurements 77d89188-4373-4ea8-8243-99b002752c83 with parameters: dry_run=True; nb_targets=2; nb_vps=4.
INFO:root:measurement : 77d89188-4373-4ea8-8243-99b002752c83 done


## retrieve prefix probing results 

In [2]:
#measurement_results = get_measurements_from_tag(str(measurement_config["UUID"]))
measurement_results = get_measurements_from_tag("d39983d6-8261-412f-8940-609f0a7b429e")
# test that everything is alright
print(f"nb measurements retrieved: {len(measurement_results)}")

for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10:
        break
    print(target_addr, ":", len(results))

# save results
dump_json(measurement_results, PREFIX_ANCHOR_VP)

nb measurements retrieved: 6
103.160.63.5 : 4
103.160.63.1 : 4
103.160.63.2 : 3
193.49.43.13 : 4
193.49.43.2 : 4
193.49.43.10 : 4


# Tier 2: probing each target

## measure targets

In [29]:
dry_run = False
measurement_uuid = uuid.uuid4()

# measurement configuration for retrieval
measurement_config = {
    "UUID": str(measurement_uuid),
    "is_dry_run": dry_run,
    "description": "measurement towards all anchors with all anchors as vps",
    "type": "target",
    "af": 4,
    "targets": targets,
    "vps": vps,
}

# save measurement configuration before starting measurement
config_file = MEASUREMENT_CONFIG_PATH / f"{measurement_config['UUID']}.json"
dump_json(measurement_config, config_file)

pinger = PING(RIPE_CREDENTIALS)


try:
    cached_results = load_json(TARGET_ANCHOR_VP)
    logging.info(
        f"initial length targets: {len(targets)}, cached measurements : {len(cached_results)}"
    )

    targets = list(
        set(targets).difference(set(cached_results)))

    logging.info(
        f"after removing cached: {len(targets)}, cached measurements : {len(cached_results)}"
    )
except FileNotFoundError:
    logging.info("No cached results available")
    pass

logging.info(
        f"Starting measurements {str(measurement_uuid)} with parameters: dry_run={dry_run}; nb_targets={len(targets)}; nb_vps={len(vps)}."
    )

# measurement for 3 targets in every target prefixes
measurement_config["ids"], measurement_config["start_time"], measurement_config["end_time"] = pinger.ping_by_target(
    targets=targets,
    vps=vps,
    tag=measurement_uuid,
    dry_run=dry_run
)

# save measurement configuration after measurements
dump_json(measurement_config, config_file)

INFO:root:No cached results available
INFO:root:Starting measurements 2820356f-1f75-48e3-ae0b-710e7e88b40e with parameters: dry_run=True; nb_targets=2; nb_vps=4.
INFO:root:measurement : 2820356f-1f75-48e3-ae0b-710e7e88b40e done


## retrieve traget probing results

In [3]:
#measurement_results = get_measurements_from_tag(str(measurement_config["UUID"]))
measurement_results = get_measurements_from_tag("baa3df49-5118-4b3c-86bb-f35d96382304")
# test that everything is alright
print(f"nb measurements retrieved: {len(measurement_results)}")

for i, (target_addr, results) in enumerate(measurement_results.items()):
    if i > 10:
        break
    print(target_addr, ":", len(results))

# save results
dump_json( measurement_results, TARGET_ANCHOR_VP)

nb measurements retrieved: 2
193.49.43.151 : 4
103.160.63.222 : 4
